This file contains code to resize the images from 1440x1616 to 512x512, which is frequently used for U-net training

In [3]:
from PIL import Image
import numpy as np
from tqdm import tqdm
import os
import glob

Running resizing and saving the images in a loop

In [ ]:
import os
import glob
from PIL import Image
import numpy as np

def resize_with_padding(img, target_size=(512, 512), color=(0, 0, 0)):
    """Resize an image to fit within target_size while maintaining aspect ratio and add padding."""
    # Get current dimensions
    width, height = img.size
    
    # Calculate scaling factor to maintain aspect ratio
    ratio = min(target_size[0] / width, target_size[1] / height)
    new_size = (int(width * ratio), int(height * ratio))
    
    # Resize image
    if img.mode == 'RGB':
        resized_img = img.resize(new_size, Image.Resampling.LANCZOS)
    else:  # For masks, use NEAREST to preserve binary values
        resized_img = img.resize(new_size, Image.Resampling.NEAREST)
    
    # Create new image with padding
    if img.mode == 'RGB':
        new_img = Image.new("RGB", target_size, color)
    else:  # For masks (grayscale/binary)
        new_img = Image.new("L", target_size, 0)
    
    # Paste resized image in center
    paste_position = ((target_size[0] - new_size[0]) // 2, 
                       (target_size[1] - new_size[1]) // 2)
    new_img.paste(resized_img, paste_position)
    
    return new_img

def process_dataset(base_dir="C:\\Users\\tania\\PythonProjects\\keylife-ng-pv\\ml_project_2\\data\\u-net-data\\", target_size=(512, 512)):
    # Define directories
    image_dir = os.path.join(base_dir, "training")
    mask_dir = os.path.join(base_dir, "training-labels")
    
    output_image_dir = os.path.join(base_dir, "512_tiff", "training")
    output_mask_dir = os.path.join(base_dir, "512_tiff", "training-labels")
    
    # Create output directories if they don't exist
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)
    
    # Get all image files
    image_files = glob.glob(os.path.join(image_dir, "*.jpg"))
    mask_files = glob.glob(os.path.join(mask_dir, "*.png"))
    
    
    # Process each image and its corresponding mask
    for mask_path in mask_files:
        try:
            # Get the base filename without extension
            base_filename = os.path.basename(mask_path).rsplit('_mask.', 1)[0]
            
            # Construct mask path
            image_path = os.path.join(image_dir, f"{base_filename}.jpg")
            
            # Skip if mask doesn't exist
            if not os.path.exists(mask_path):
                print(f"Warning: Mask not found for {base_filename}. Skipping.")
                continue
            
            # Load and process image
            image = Image.open(image_path)
            if image.mode != 'RGB':
                image = image.convert('RGB')
            resized_image = resize_with_padding(image, target_size)
            
            # Load and process mask
            mask = Image.open(mask_path)
            if mask.mode != 'L':
                mask = mask.convert('L')
            resized_mask = resize_with_padding(mask, target_size)
            
            # Save as TIFF
            output_image_path = os.path.join(output_image_dir, f"{base_filename}.jpg")
            output_mask_path = os.path.join(output_mask_dir, f"{base_filename}_mask.png")
            
            resized_image.save(output_image_path, format='TIFF', compression='tiff_lzw')
            resized_mask.save(output_mask_path, format='TIFF', compression='tiff_lzw')
            
            print(f"Processed {base_filename}")
            
        except Exception as e:
            print(f"Error processing {os.path.basename(image_path)}: {e}")
    
    print(f"Processing complete. Resized images and masks saved to {os.path.join(base_dir, '512_tiff')}")


In [3]:
process_dataset(target_size=(512, 512))

Processed 20250527212624
Processed 20250527212634
Processed 20250527212639
Processed 20250527212643
Processed 20250527212651
Processed 20250527212654
Processed 20250527212658
Processed 20250527212726
Processed 20250527212731
Processed 20250527212735
Processed 20250527212749
Processed 20250527212753
Processed 20250527212756
Processed 20250527212800
Processed 20250527212803
Processed 20250527212807
Processed 20250527212915
Processed 20250527212943
Processed 20250527212947
Processed 20250527213001
Processed 20250527213020
Processed 20250527213024
Processed 20250527213048
Processed 20250527213105
Processed 20250527213123
Processed 20250527213139
Processed 20250527213208
Processed 20250527213347
Processed 20250527213402
Processed 20250527213410
Processed 20250527213426
Processed 20250527213445
Processed 20250527213635
Processed 20250527213654
Processed 20250527213658
Processed 20250527213705
Processed 20250527213729
Processed 20250527213742
Processed 20250527213809
Processed 20250527213817


In [ ]:
from collections import defaultdict

def rename_files_to_sequential(base_dir="C:\\Users\\tania\\PythonProjects\\keylife-ng-pv\\ml_project_2\\data\\u-net-data\\"):
    # Define directories
    image_dir = os.path.join(base_dir, "512_jpg", "training")
    mask_dir = os.path.join(base_dir, "512_jpg", "training-labels")
    
    # Check if directories exist
    if not os.path.exists(image_dir) or not os.path.exists(mask_dir):
        print(f"Error: One or both directories don't exist.")
        return
    
    # Get all tiff files
    image_files = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
    mask_files = sorted(glob.glob(os.path.join(mask_dir, "*_mask.jpg")))
    
    # Create a mapping between image base names and their full paths
    image_map = defaultdict(dict)
    for img_path in image_files:
        base_name = os.path.basename(img_path).rsplit('.', 1)[0]
        image_map[base_name]['image'] = img_path
    
    for mask_path in mask_files:
        base_name = os.path.basename(mask_path).rsplit('_mask.', 1)[0]
        image_map[base_name]['mask'] = mask_path
    
    # Only keep pairs where both image and mask exist
    complete_pairs = {k: v for k, v in image_map.items() if 'image' in v and 'mask' in v}
    
    if not complete_pairs:
        print("No matching image-mask pairs found.")
        return
    
    print(f"Found {len(complete_pairs)} image-mask pairs to rename.")
    
    # Rename files to sequential numbers
    for idx, (base_name, paths) in enumerate(sorted(complete_pairs.items())):
        # New filenames
        new_image_name = f"{idx}.tif"
        new_mask_name = f"{idx}.tif"
        
        # Full paths for new files
        new_image_path = os.path.join(image_dir, new_image_name)
        new_mask_path = os.path.join(mask_dir, new_mask_name)
        
        # Rename files
        try:
            os.rename(paths['image'], new_image_path)
            os.rename(paths['mask'], new_mask_path)
            print(f"Renamed pair {idx}: {base_name}")
        except Exception as e:
            print(f"Error renaming {base_name}: {e}")
    
    print(f"Renaming complete. Files have been renamed to sequential numbers in both directories.")

In [5]:
rename_files_to_sequential()

Found 110 image-mask pairs to rename.
Renamed pair 0: 20250527212624
Renamed pair 1: 20250527212634
Renamed pair 2: 20250527212639
Renamed pair 3: 20250527212643
Renamed pair 4: 20250527212651
Renamed pair 5: 20250527212654
Renamed pair 6: 20250527212658
Renamed pair 7: 20250527212726
Renamed pair 8: 20250527212731
Renamed pair 9: 20250527212735
Renamed pair 10: 20250527212749
Renamed pair 11: 20250527212753
Renamed pair 12: 20250527212756
Renamed pair 13: 20250527212800
Renamed pair 14: 20250527212803
Renamed pair 15: 20250527212807
Renamed pair 16: 20250527212915
Renamed pair 17: 20250527212943
Renamed pair 18: 20250527212947
Renamed pair 19: 20250527213001
Renamed pair 20: 20250527213020
Renamed pair 21: 20250527213024
Renamed pair 22: 20250527213048
Renamed pair 23: 20250527213105
Renamed pair 24: 20250527213123
Renamed pair 25: 20250527213139
Renamed pair 26: 20250527213208
Renamed pair 27: 20250527213347
Renamed pair 28: 20250527213402
Renamed pair 29: 20250527213410
Renamed pair

DO IT FOR JPG FILES

In [4]:

def resize_with_padding(img, target_size=(512, 512), color=(0, 0, 0)):
    """Resize an image to fit within target_size while maintaining aspect ratio and add padding."""
    # Get current dimensions
    width, height = img.size
    
    # Calculate scaling factor to maintain aspect ratio
    ratio = min(target_size[0] / width, target_size[1] / height)
    new_size = (int(width * ratio), int(height * ratio))
    
    # Resize image
    if img.mode == 'RGB':
        resized_img = img.resize(new_size, Image.Resampling.LANCZOS)
    else:  # For masks, use NEAREST to preserve binary values
        resized_img = img.resize(new_size, Image.Resampling.NEAREST)
    
    # Create new image with padding
    if img.mode == 'RGB':
        new_img = Image.new("RGB", target_size, color)
    else:  # For masks (grayscale/binary)
        new_img = Image.new("L", target_size, 0)
    
    # Paste resized image in center
    paste_position = ((target_size[0] - new_size[0]) // 2, 
                       (target_size[1] - new_size[1]) // 2)
    new_img.paste(resized_img, paste_position)
    
    return new_img

def process_dataset(base_dir="C:\\Users\\tania\\PythonProjects\\keylife-ng-pv\\ml_project_2\\data\\u-net-data\\", target_size=(512, 512)):
    # Define directories
    image_dir = os.path.join(base_dir, "training")
    mask_dir = os.path.join(base_dir, "training-labels")
    
    output_image_dir = os.path.join(base_dir, "512_jpg", "training")
    output_mask_dir = os.path.join(base_dir, "512_jpg", "training-labels")
    
    # Create output directories if they don't exist
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)
    
    # Get all image files
    image_files = glob.glob(os.path.join(image_dir, "*.jpg"))
    
    # Process each image and its corresponding mask
    for image_path in image_files:
        try:
            # Get the base filename without extension
            base_filename = os.path.basename(image_path).rsplit('.', 1)[0]
            
            # Construct mask path
            mask_path = os.path.join(mask_dir, f"{base_filename}_mask.png")
            
            # Skip if mask doesn't exist
            if not os.path.exists(mask_path):
                print(f"Warning: Mask not found for {base_filename}. Skipping.")
                continue
            
            # Load and process image
            image = Image.open(image_path)
            if image.mode != 'RGB':
                image = image.convert('RGB')
            resized_image = resize_with_padding(image, target_size)
            
            # Load and process mask
            mask = Image.open(mask_path)
            if mask.mode != 'L':
                mask = mask.convert('L')
            resized_mask = resize_with_padding(mask, target_size)
            
            # Save as JPG
            output_image_path = os.path.join(output_image_dir, f"{base_filename}.jpg")
            output_mask_path = os.path.join(output_mask_dir, f"{base_filename}_mask.jpg")
            
            # Save image as JPG with high quality (95)
            resized_image.save(output_image_path, format='JPEG', quality=95)
            
            # Save mask as JPG - note that JPG is lossy and not ideal for binary masks
            # but we'll use high quality to minimize issues
            resized_mask.save(output_mask_path, format='JPEG', quality=100)
            
            print(f"Processed {base_filename}")
            
        except Exception as e:
            print(f"Error processing {os.path.basename(image_path)}: {e}")
    
    print(f"Processing complete. Resized images and masks saved to {os.path.join(base_dir, '512_jpg')}")


In [5]:
process_dataset(target_size=(512, 512))

Processed 20250527212624
Processed 20250527212634
Processed 20250527212639
Processed 20250527212643
Processed 20250527212651
Processed 20250527212654
Processed 20250527212658
Processed 20250527212726
Processed 20250527212731
Processed 20250527212735
Processed 20250527212749
Processed 20250527212753
Processed 20250527212756
Processed 20250527212800
Processed 20250527212803
Processed 20250527212807
Processed 20250527212915
Processed 20250527212943
Processed 20250527212947
Processed 20250527213001
Processed 20250527213020
Processed 20250527213024
Processed 20250527213048
Processed 20250527213105
Processed 20250527213123
Processed 20250527213139
Processed 20250527213208
Processed 20250527213347
Processed 20250527213402
Processed 20250527213410
Processed 20250527213426
Processed 20250527213445
Processed 20250527213635
Processed 20250527213654
Processed 20250527213658
Processed 20250527213705
Processed 20250527213729
Processed 20250527213742
Processed 20250527213809
Processed 20250527213817
